In [1]:
import cv2
import matplotlib.pyplot as plt
from PIL import Image
import os
import numpy as np
from typing import Callable
import skimage
from skimage.color import rgb2hsv
from skimage import io, util
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

In [2]:
# Define path
path0 = os.path.join("cropped_0.jpg")
path1 = os.path.join("cropped_1.jpg")
path2 = os.path.join("cropped_2.jpg")


# Check if folder and image exist
assert os.path.exists(path0), "Image not found, please check directory structure"
assert os.path.exists(path1), "Image not found, please check directory structure"
assert os.path.exists(path2), "Image not found, please check directory structure"

# Load image
image_0= np.array(Image.open(path0))
image_1= np.array(Image.open(path1))
image_2= np.array(Image.open(path2))


In [3]:
# Step 1: Define the path to your dataset folder
data_dir = 'my_dataset'

# Step 2: Define Transforms (Optional)
transform = transforms.Compose([
    transforms.Resize((128, 128)),  # Resize images to a fixed size if needed
    transforms.ToTensor(),           # Convert PIL image to tensor
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Normalize images
])

# Step 3: Create Dataset Object
dataset = ImageFolder(root=data_dir, transform=transform)

# Step 4: Instantiate DataLoader
batch_size = 64
shuffle = True  # Shuffle data during training
num_workers = 4  # Number of subprocesses to use for data loading
trainloader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, num_workers=num_workers)


In [4]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        # self.conv1 = nn.Conv2d(3, 6, 5)
        # self.pool = nn.MaxPool2d(2, 2)
        # self.conv2 = nn.Conv2d(6, 16, 5)
        # self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc1 = nn.Linear(128*128*3, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 3)

    def forward(self, x):
        # x = self.pool(F.relu(self.conv1(x)))
        # x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net()

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.005, momentum=0.9)

epochs = 15
for epoch in range(epochs):  # loop over the dataset multiple times
    print("epoch ", epoch)

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        print("loss: ", loss.item())
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')

epoch  0
loss:  1.0993071794509888
epoch  1
loss:  1.072140097618103
epoch  2
loss:  1.0511088371276855
epoch  3
loss:  1.0021594762802124
epoch  4
loss:  0.9325926303863525
epoch  5
loss:  0.8590817451477051
epoch  6
loss:  0.7609049677848816
epoch  7
loss:  0.6482694745063782
epoch  8
loss:  0.5408826470375061
epoch  9
loss:  0.4241959750652313
epoch  10
loss:  0.31729331612586975
epoch  11
loss:  0.22149424254894257
epoch  12
loss:  0.1475239247083664
epoch  13
loss:  0.09607163816690445
epoch  14
loss:  0.0617181695997715
Finished Training


In [12]:
#checking the labels

for batch_idx, (images, labels) in enumerate(trainloader):
    # for i in range(len(images)):
    outputs = net.forward(images)
    loss = criterion(outputs, labels)
    predictions = torch.argmax(outputs, dim=1)
    print("predicted labels: ", predictions)
    print("true labels: ", labels)
    num_points = len(labels)
    correct =sum(predictions == labels)
    accuracy = correct / num_points * 100
    print("accuracy: ", accuracy, "%")


predicted labels:  tensor([1, 2, 0])
true labels:  tensor([1, 2, 0])
accuracy:  tensor(100.) %
